In [1]:
import pandas as pd
import glob
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
# set column width to be able to see full text
pd.set_option('display.max_colwidth', 100)

model_rankings = 'monoT5'

base_path = f'../indexing/results/{model_rankings}/'
# all_model_dirs = glob.glob(f'../indexing/results/{model_rankings}/*')
all_model_csvs = glob.glob(f'{base_path}/*.csv') 
plot_path = f'./plots'
if not os.path.exists(plot_path):
    os.makedirs(plot_path)
number_of_answers = 10

model_dfs = []
for model_csv in all_model_csvs:
    df = pd.read_csv(model_csv)
    model_name = model_csv.split("/")[-1].split(".")[0].replace("_rankings", "")
    df["model"] = model_name
    model_dfs.append(df)

all_model_df = pd.concat(model_dfs, ignore_index=True)
all_model_df['is_natural_question'] = all_model_df['query'].str.endswith('?')
# replace na answers with empty string
all_model_df['text'] = all_model_df['text'].fillna('')

In [2]:
import os

# Directory path
dir_path = 'rankings'

def get_top_k_from_rankings_for_qid(qid, k):
    file_path = os.path.join(dir_path, f'query_{qid}_rankings.csv')
    qid_df = pd.read_csv(file_path)
    qid_df = qid_df.head(k)
    return qid_df

# for model, get the k highest scored answers for qid
def get_top_k_answers(df, model, qid, k):
    model_df = df[df['model'] == model]
    qid_df = model_df[model_df['qid'] == qid]
    top_k_df = qid_df.sort_values(by=['score'], ascending=False).head(k)
    return top_k_df

In [3]:
# get top 1 answer of each model and mix them with the documents from the original ranking

def get_top_1_answers_with_original_ranking(df, qid, k_original):
    top_1_answers = []
    for model in df['model'].unique():
        top_k_answers_model = get_top_k_answers(df, model, qid, 1)
        # combine docno with prompt and answer_number cols
        top_k_answers_model['docno'] = top_k_answers_model['docno'] + '_' + top_k_answers_model['prompt'] + "_" + top_k_answers_model['answer_number'].astype(str)
        top_1_answers.append(top_k_answers_model)
    top_1_answers = pd.concat(top_1_answers)
    top_k_documents = get_top_k_from_rankings_for_qid(qid, k_original)
    # drop all rows except docno, text, score in both dataframes
    top_1_answers = top_1_answers[['docno', 'text', 'score']]
    top_k_documents = top_k_documents[['docno', 'text', 'score']]
    # combine both dataframes
    top_k_answers_with_original_ranking = pd.concat([top_1_answers, top_k_documents])
    # sort by score
    top_k_answers_with_original_ranking = top_k_answers_with_original_ranking.sort_values(by=['score'], ascending=False)
    # limit text length to 512 words
    top_k_answers_with_original_ranking['text'] = top_k_answers_with_original_ranking['text'].str.split().str[:512].str.join(" ")
    return top_k_answers_with_original_ranking

# only use rows with prompt multimedqa
all_model_df = all_model_df[all_model_df['prompt'] == 'multimedqa']
for qid in all_model_df['qid'].unique():
    qid_df = get_top_1_answers_with_original_ranking(all_model_df, qid, 5)
    # make directory if not exists
    qid_dir = f'./top_1_answers_with_original_ranking'
    if not os.path.exists(qid_dir):
        os.makedirs(qid_dir)
    qid_df.to_csv(f'./{qid_dir}/qid_{qid}.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'rankings/query_112_rankings.csv'